In [ ]:
# !jupyter kernelspec list
# !jupyter notebook --NotebookApp.kernel_name=python3

In [83]:
# !pip install -q pybotters numpy pandas
!pip install -q python-dotenv

In [ ]:
import pybotters
import pandas as pd
from pandas import DataFrame


## Idea
* バブル相場ではどうせ価格は上がるんだから、一時的に下げた後のお得なところで買えば勝てるよね！

## 設計構想

* 直近N分の価格取得
  * 最低価格を計算


* 

## 参考

[バブル相場用の回転ボットのコンセプトとQuantZoneロジック(追記あり)](https://note.com/hht/n/n63022edc4610)

* 「レバレッジL倍で、直近N分高値から、R %価格が下がった位置にエントリー指値」
* 「直近N分安値から、R %価格が上がったら位置に決済指値」

In [1]:
import pybotters

In [106]:
class Exchange():
    """
    取引所と直接APIのやり取りを行う
    """
    def __init__(self, client):
        self.client = client
        pass
    
    async def get_klines(self, pair_symbol:str, interval:str='60', limit:str='5') -> pd.DataFrame:
        """
        /v5/market/kline
        ローソク足(kline)を取得
        """

        async with self.client.get(
            f"/v5/market/kline?category=linear&interval=60&symbol={pair_symbol}&interval={interval}&limit={limit}"
        ) as resp:
            content = await resp.json()

        klines = content['result']['list']
        klines = pd.DataFrame(klines)
        klines.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover']

        return klines

    async def get_latest_price(self, pair_symbol:str):
        """"""""
        klines = await self.get_klines(pair_symbol, interval='1', limit='1')
        latest_price = float(klines['close'].values[-1])
        return latest_price


    async def get_symbol_info(self):
        """銘柄の情報を取得"""
        pass

    async def get_position(self):
        async with self.client.get("/v5/position/list?category=linear&settleCoin=USDT") as resp:
            content = await resp.json()
            symbol_list = [{'symbol': json['symbol'], 'side': json['side'], 'size': json['size']} for json in content['result']['list']]
            return symbol_list

    async def create_order(self, pair_symbol, qty, side='Buy', price=None, reduce_only=False):
        data = data={
                'category': "linear",
                'symbol': pair_symbol,
                'side': side,
                'orderType': 'Limit',
                'price': price,
                'qty': qty,
                'reduceOnly': reduce_only,
        }
        print(data)
        async with self.client.post(
            "/v5/order/create",
            data=data) as resp:
            content = await resp.json()
            print(content)
            return content['retMsg']

    

In [107]:
# class DataFetcher():
#     """
#     Exchangeを介して、取引所からデータを取得する
#     """
#     def __init__(self, exchange):
#         self.exchange = exchange
#         pass

#     async def get_klines(self):
#         klines = await self.exchange.get_klines()
#         klines = pd.DataFrame(klines)
#         klines.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover']

#     def 


In [108]:


class TradingStrategy():
    """
    ロジックをもとに売買に関わるシグナルを作成する
    """
    def __init__(self, exchange: Exchange, trading_volume: float=10, rate_of_drop:float=0.2, rate_of_pump:float=0.2):
        self.exchange = exchange
        self.trading_volume = trading_volume
        self.rate_of_drop = rate_of_drop
        self.rate_of_pump = rate_of_pump
    
    def calc_position_size(self):

        pass

    async def calc_buy_price(self, pair_symbol:str, interval:str=60) -> float:
        klines:DataFrame = await self.exchange.get_klines(pair_symbol=pair_symbol, interval=interval, limit='5')
        high:float = klines['high'].astype(float).values
        highest_price:float = high.max()
        buy_price:float = highest_price * (1 - self.rate_of_drop)
        # レバレッジ部分は追加実装する必要あり
        return buy_price

    async def calc_sell_price(self, pair_symbol:str, interval:str=60) -> float:
        klines:DataFrame = await self.exchange.get_klines(pair_symbol=pair_symbol, interval=interval, limit='5')
        low:float = klines['low'].astype(float).values
        lowest_price:float = low.min()
        sell_price:float = lowest_price * (1 + self.rate_of_pump)
        return sell_price

In [113]:
class KaitenBot():
    """
    ロジックに沿ってトレードを行う最上位クラス
    """
    def __init__(self, client):
        self.exchange = Exchange(client)
        self.trading_strategy = TradingStrategy(self.exchange)
    
    async def run(self):
        pair_symbol = 'TRBUSDT'
        buy_price = await self.trading_strategy.calc_buy_price(pair_symbol)
        ret_msg = await self.exchange.create_order(pair_symbol=pair_symbol, qty='0.01', side='Buy', price=str(int(buy_price)), reduce_only=False)
        print(ret_msg)
        

In [117]:
import os
from dotenv import load_dotenv

load_dotenv('.env')

apis = {
    'bybit': [os.getenv('API_KEY'), os.getenv('API_SECRET')]
}

async with pybotters.Client(apis=apis, base_url='https://api.bybit.com') as client:
    kaiten_bot = KaitenBot(client)
    await kaiten_bot.run()


{'category': 'linear', 'symbol': 'TRBUSDT', 'side': 'Buy', 'orderType': 'Limit', 'price': '145', 'qty': '0.01', 'reduceOnly': False}
{'retCode': 0, 'retMsg': 'OK', 'result': {'orderId': '15200b01-d016-4b82-a37c-9b9d37a7a0d0', 'orderLinkId': ''}, 'retExtInfo': {}, 'time': 1704266719173}
OK


In [116]:
import os
from dotenv import load_dotenv

load_dotenv('.env')

apis = {
    'bybit': [os.getenv('API_KEY'), os.getenv('API_SECRET')]
}

async with pybotters.Client(apis=apis, base_url='https://api.bybit.com') as client:
    exchange = Exchange(client)
    # klines = await exchange.get_klines()
    # print(klines)
    latest_price = await exchange.get_latest_price()
    print(latest_price)
    # position = await exchange.get_position()
    # print(position)
    trading_strategy = TradingStrategy(exchange)
    buy_price = await trading_strategy.calc_buy_price()
    print(f'{buy_price=}, {latest_price * (1 - trading_strategy.rate_of_drop)=}')


TypeError: Exchange.get_latest_price() missing 1 required positional argument: 'pair_symbol'

In [ ]:
# class DataFetcher():
#     """
#     Exchangeを介して、取引所からデータを取得する
#     """
#     def __init__(self, exchange):
#         self.exchange = exchange
#         pass

#     async def get_klines(self):
#         klines = await self.exchange.get_klines()
#         klines = pd.DataFrame(klines)
#         klines.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover']

#     def 


In [78]:
buy_price

36340.8